In [1]:
fws = ['tflow','ptorch']

In [2]:
fw = fws[0]

In [3]:
fw

'tflow'

In [4]:
import organise_test as org

In [5]:
lenet1 = org.give_df('LeNet_1.{}'.format(fw))
lenet5 = org.give_df('LeNet_5.{}'.format(fw))
myalexnet = org.give_df('My_AlexNet.{}'.format(fw))
myvgg11 = org.give_df('My_VGG_11.{}'.format(fw))

In [6]:
import sys
sys.path.append('../')
import lib
if fw=='tflow':
    import tflow as tools
    import execute.tflow_models as tm
else:
    import ptorch as tools
    import execute.ptorch_models as tm

In [7]:
tools

<module 'tflow' from '/home/vlassis/Desktop/Diploma/gits/estimate/tflow.py'>

In [8]:
tm

<module 'execute.tflow_models' from '../execute/tflow_models.py'>

In [9]:
Regs = [None]+[lib.Reg_N(fw,node) for node in [1,2,3]]

In [10]:
class Pred:
    def __init__(self,Regs):
        self.Regs = Regs    
    
    def create(self,the_df,Model):
        def give(epochs,ds,numf,channels,batch,nodes):
            M = Model()
            model = M.create()
            layers = tools.extract(model, 2, batch, numf, channels)
            total = self.Regs[nodes].predict(layers, epochs, ds, batch)

            return total
        
        def pred(x):
            time = give(int(x['Epochs']),int(x['Dataset size']),int(x['Number of features']),
                        int(x['Channels']),int(x['Batch size']),int(x['Nodes']))
            return time
            
        the_df['Predicted(s)'] = the_df.apply(pred,axis=1)
        the_df['Difference(s)'] = abs(the_df['Predicted(s)']-the_df['Time (s)'])
        the_df['Error%'] = round(100*abs(the_df['Predicted(s)']-the_df['Time (s)'])/the_df['Time (s)'],1)
        
#         return the_df.drop(columns=['Epochs','Number of features', 'Channels'])
        return the_df.drop(columns=['Dataset size','Epochs','Number of features', 'Channels'])

In [11]:
def save(df,title):
    the = {
        'tflow':'TensorFlow',
        'ptorch':'PyTorch'
    }
    with open('tables/{}.tex'.format(fw), 'a') as file:
#         df.to_latex(file,caption='{} - {}'.format(the[fw],title), 
#                     label='fig:{}_{}'.format(fw,title),
#                     index=False)
        tab = df.to_latex(caption='{}-{}'.format(the[fw],title), 
                    label='fig:{}_{}'.format(fw,title),
                    index=False)

        [pa,pb] = tab.split('\\begin{tabular}',1)
        [pb1,pb2] = pb.split('\\end{tabular}',1)
        
        pa += '\\scalebox{0.8}{\n'
        pb2 = '\n}' + pb2
        
        res = pa+'\\begin{tabular}'+pb1+'\\end{tabular}'+pb2
        file.write(res+'\n\n')
        
    with open('tables/{}.csv'.format(fw), 'a') as file:
        df.to_csv(file,index=False)

In [12]:
def update(mean,var):
    the = {
        'tflow':'TensorFlow',
        'ptorch':'PyTorch'
    }
    with open('tables/{}.tex'.format(fw), 'a') as file:
        file.write('Μέσος όρος σφάλματος: {}\\% \n\nΤυπική απόκλιση σφάλματος: {}\\% \n\n'.format(mean, var))

In [13]:
p = Pred(Regs)

In [14]:
df_lenet1 = p.create(lenet1,tm.LeNet_1)

In [15]:
df_lenet1

,Batch size,Nodes,Time (s),Predicted(s),Difference(s),Error%
0,16.0,1.0,5.552860,6.121298,0.568437,10.2
1,16.0,2.0,2.776430,3.040068,0.263638,9.5
2,16.0,3.0,1.850953,2.027937,0.176983,9.6
3,32.0,1.0,5.129190,5.297491,0.168302,3.3
4,32.0,2.0,2.564595,2.635240,0.070645,2.8
5,32.0,3.0,1.709730,1.758231,0.048501,2.8
6,64.0,1.0,4.768607,4.898695,0.130087,2.7
7,64.0,2.0,2.384304,2.442983,0.058679,2.5
8,64.0,3.0,1.589536,1.625372,0.035836,2.3
9,128.0,1.0,3.726668,4.398470,0.671802,18.0


In [16]:
save(df_lenet1.head(12),'LeNet1 (4608)')

In [17]:
save(df_lenet1.tail(12),'LeNet1 (36864)')

In [18]:
df_lenet5 = p.create(lenet5,tm.LeNet_5)

In [19]:
df_lenet5

,Batch size,Nodes,Time (s),Predicted(s),Difference(s),Error%
0,16.0,1.0,7.766443,9.548830,1.782388,22.9
1,16.0,2.0,3.883221,4.636334,0.753112,19.4
2,16.0,3.0,2.588814,3.145485,0.556670,21.5
3,32.0,1.0,6.550888,7.474317,0.923430,14.1
4,32.0,2.0,3.275444,3.680692,0.405249,12.4
5,32.0,3.0,2.183629,2.487318,0.303689,13.9
6,64.0,1.0,6.107675,7.457516,1.349841,22.1
7,64.0,2.0,3.053838,3.546394,0.492556,16.1
8,64.0,3.0,2.035892,2.583921,0.548029,26.9
9,128.0,1.0,5.306523,7.711742,2.405220,45.3


In [20]:
save(df_lenet5.head(12),'LeNet5 (4608)')

In [21]:
save(df_lenet5.tail(12),'LeNet5 (36864)')

In [22]:
df_myvgg1 = p.create(myvgg11,tm.My_VGG_11)

In [23]:
df_myvgg1

,Batch size,Nodes,Time (s),Predicted(s),Difference(s),Error%
0,16.0,1.0,332.963549,192.273759,140.689791,42.3
1,16.0,2.0,166.481775,95.909004,70.572771,42.4
2,16.0,3.0,110.987850,63.155135,47.832714,43.1
3,32.0,1.0,304.234759,196.238957,107.995802,35.5
4,32.0,2.0,152.117379,97.471828,54.645552,35.9
5,32.0,3.0,101.411586,65.577916,35.833671,35.3
6,64.0,1.0,257.896816,177.123975,80.772841,31.3
7,64.0,2.0,128.948408,89.671676,39.276732,30.5
8,64.0,3.0,85.965605,59.696809,26.268796,30.6
9,128.0,1.0,238.213484,166.316667,71.896817,30.2


In [24]:
save(df_myvgg1,'VGG11 (4608)')

In [25]:
def to_vec(*args):
    vec = []
    for elem in args:
        vec += list(elem)
    return vec

In [26]:
errs = to_vec(df_lenet1['Error%'],df_lenet5['Error%'],df_myvgg1['Error%'])

In [27]:
import statistics as st

In [28]:
mean = st.mean(errs)

In [29]:
mean

21.696666666666665

In [30]:
var = st.stdev(errs)

In [31]:
var

14.861735266826782

In [32]:
update(round(mean,2),round(var,2))

In [33]:
# df_myalexnet = p.create(myalexnet,tm.My_AlexNet)

# df_myalexnet

# save(df_myalexnet,'AlexNet')